In [1]:
import h5py
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Resizing, Dropout, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Input, Reshape, Conv2DTranspose, Cropping2D

from keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from tools import define_cnn, extract_features, build_model, predict_with_model, set_seed, build_model_and_predict, extract_features_2
import warnings
warnings.filterwarnings('ignore')

c:\Users\jakub\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# load clean gdp data, keep only year, region and real_gdp columns
poland = pd.read_csv("data/tabular_data_poland.csv")
log_bin_columns = ["allangle_snow_free" + "_log_" + str(i) for i in range(1, 11)]

# get the data for 2021, 2022 and before 2022
poland_2022 = poland[poland["year"] == 2022]
poland = poland[poland["year"].astype(int) < 2022]
poland_2021 = poland[poland["year"] == 2021]
poland_2021.reset_index(drop=True, inplace=True)
poland_2022.reset_index(drop=True, inplace=True)
poland.reset_index(drop=True, inplace=True)

# Initialise a three dimensional array to store the images with the shape (number of images, height, width, channels)
X = np.zeros((len(poland), 609, 911, 1))
y = np.zeros(len(poland))

# load the images
for i in range(len(poland)):

    # get year, region, and gdp
    year = poland["year"][i]
    region = poland["region"][i]
    gdp_value = poland["real_gdp"][i]

    # load the image
    file_name = f"{year}_{region}_hq.h5"
    file_path = f"data/annual_region_images/{file_name}"
    
    with h5py.File(file_path, 'r') as annual_region:
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]

    # add the values
    y[i] = gdp_value
    X[i, :, :, 0] = allangle_snow_free

# normalise the images and gdp data
maximum_x = X.max()
X = X / maximum_x

maximum_y = y.max()
y = y / maximum_y

# get indices for observations with  year = 2021: this is the test set
test_indices = np.where(poland["year"] == 2021)[0]
train_indices = np.where(poland["year"] != 2021)[0]

# get the train and test sets
X_train, y_train, X_test, y_test = X[train_indices], y[train_indices], X[test_indices], y[test_indices]

# get the prediction data
X_pred = np.zeros((len(poland_2022),  609, 911, 1))
for i in range(len(poland_2022)):

    year = poland_2022["year"][i]
    region = poland_2022["region"][i]

    file_name = f"{year}_{region}_hq.h5"
    file_path = f"data/annual_region_images/{file_name}"

    with h5py.File(file_path, 'r') as annual_region:
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]
    
    X_pred[i, :, :, 0] = allangle_snow_free

X_pred = X_pred / maximum_x

In [3]:
poland = pd.read_csv("data/tabular_data_poland.csv")
poland = poland[poland["year"] < 2023]
poland_sum = poland[['year', 'region', 'allangle_snow_free_hq_sum', 'allangle_snow_free_hq_mean'] + log_bin_columns]

model = define_cnn(n_features = 4, n_conv = 2, n_dense = 4, input_shape = (609, 911, 1), res_x = 305, res_y = 455)
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])
model.fit(X, y, epochs=50, batch_size=64, validation_split=0.2)

poland_stage_2, poland_pred = extract_features_2(model, poland, 2022, X, X_pred, 4)
selected_columns =  ["feature_" + str(i) for i in range(1, 5)] + ["allangle_snow_free_hq_sum", "allangle_snow_free_hq_mean"] + log_bin_columns

param_grid_rf = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [5, 10, 15, 20],  # Maximum depth of the tree
        'min_samples_split': [2, 4, 6, 8],  # Minimum number of samples required to split an internal node
        'min_samples_leaf': [2, 4, 6, 8],  # Minimum number of samples required to be at a leaf node
        'random_state': [0],  # Ensures reproducibility
}

full_data = pd.concat([poland_stage_2, poland_pred])
full_data.sort_values(by=['region', 'year'], inplace=True)
full_data = pd.merge(full_data, poland_sum, on=['year', 'region'])
full_data_diff = full_data.groupby('region').diff()
full_data_diff['region'] = full_data['region']
full_data_diff['year'] = full_data['year']
full_data_diff.reset_index(drop=True, inplace=True)
full_data_diff = full_data_diff[full_data_diff["year"] != 2012]

train_data_diff = full_data_diff[full_data_diff["year"] != 2022]
pred_data_diff = full_data_diff[full_data_diff["year"] == 2022]
pred_data_diff.reset_index(drop=True, inplace=True)
train_data_diff.reset_index(drop=True, inplace=True)

_, y_pred, best_params = predict_with_model(train_data_diff, pred_data_diff, selected_columns, "random_forest", param_grid_rf, log_transform = False, scale = False)




Epoch 1/50


2/2 [==============================] - 7s 2s/step - loss: 0.1000 - mae: 0.2369 - val_loss: 0.0366 - val_mae: 0.1558
Epoch 2/50
2/2 [==============================] - 1s 611ms/step - loss: 0.0699 - mae: 0.1809 - val_loss: 0.0183 - val_mae: 0.1084
Epoch 3/50
2/2 [==============================] - 1s 598ms/step - loss: 0.0597 - mae: 0.1792 - val_loss: 0.0264 - val_mae: 0.1395
Epoch 4/50
2/2 [==============================] - 1s 610ms/step - loss: 0.0503 - mae: 0.1552 - val_loss: 0.0182 - val_mae: 0.0982
Epoch 5/50
2/2 [==============================] - 1s 652ms/step - loss: 0.0540 - mae: 0.1438 - val_loss: 0.0178 - val_mae: 0.1100
Epoch 6/50
2/2 [==============================] - 1s 652ms/step - loss: 0.0504 - mae: 0.1581 - val_loss: 0.0262 - val_mae: 0.1402
Epoch 7/50
2/2 [==============================] - 1s 778ms/step - loss: 0.0479 - mae: 0.1552 - val_loss: 0.0165 - val_mae: 0.1043
Epoch 8/50
2/2 [==============================] - 1s 667ms/step - loss: 0.0478 - mae: 0.1

In [13]:
# save the predictions
poland_2022["gdp_pred"] = poland_2021["real_gdp"] + y_pred
poland_pred = poland_2022[["year", "region", "gdp_pred", "real_gdp"]]
poland_pred.to_csv("poland_predictions.csv", index=False)
